In [99]:
import random
import math
#Paqueterías de filtrado
from nltk.corpus import stopwords
from unidecode import unidecode
import re

In [100]:
#Abrimos el texto del Quijote
#En este caso no filtraremos nada para que quede lo más parecido a un texto original. Esto claramente traerá problemas
#de optimización y no generará texto tan coherente, pero más adelante lo intentaremos filtrando signos, puntuaciones y 
#conviritendo todo el texto a minusculas.
with open("El Quijote.txt", "r", encoding="utf-8") as file:
    content = file.read()
    words = content.split()

In [101]:
# Función que recibe un texto en forma de lista y te calcula la frecuencia con la que aparecen ciertas palabras
# despues de una palabra específica
def probabilidadesTransición(words):
# P será nuestro diccionario que tendrá como llave todas las palabras disponibles en el texto
# y como valor OTRO DICCIONARIO, cuyo llave volverán a ser todas las palabras disponibles en el texto
# y como valor la probabilidad que despues de la primera palabra (primera llave) siga la segunda palabra
# (segunda llave)
    P = {}
#Recorremos cada palabra en la lista y analizamos la primera palabra y su sucesora
    for i in range(len(words)-1):
#X: presente, Y: futuro
        X = words[i]
        Y = words[i+1]
#Si x no está en el diccionario, la agregamos y agregamos como valor otro diccionario incluyendo como llave a Y y agregarle
# 1 a la frecuencia en la que aparece Y despues de X
        if P.get(X) is None:
            P[X] = {}
            P[X][Y] = 1
#Si X ya está, ahora checamos si Y está como llave en el diccionario en el valor de X
        else:
            if P[X].get(Y) is None:
#Si no está ponemos como valor 1 y si sí está le agregamos 1 a la frecuencia
                P[X][Y] = 1
            else:
                P[X][Y] += 1
#Teniendo ya el diccionario con las frecuencias, ahora toca sacar las probabilidades de que salga cada palabra
#Recorremos cada llave del diccionario y sumamos todos los valores encontrados en el diccionario de esa llave
    for i in P.keys():
        s = float(sum(P[i].values()))
#Ahora trecorremos cada llave del segundo diccionario (del valor de la primera llave) y ajustamos el valor de la frecuencia
#dividiendola entre la suma anterior y asi obteniendo una probabilidad elemento del (0,1)
        for k in P[i].keys():
            P[i][k] = P[i][k]/s
#Devolvemos la matriz de transición
    return P

In [102]:
#Creamos la "Matriz" de transición basado en el texto del Quijote
modelo = probabilidadesTransición(words)

In [103]:
#Ahora para crear el texto, despues de una palabra dada haremos lo sgiuiente.
#Primero muestrearemos una palabra de la distribución de probabilidad obtenida (la matriz o diccionario creado en la función anterior)
#utilizando el método de Metropolis-Hastings.
#Para ello, primero crearemos una función alpha que penalice las transiciones poco probables. En este caso utilizaremos el negativo del 
#logaritmo de la probabilidad de transición.
def alpha(P, current_word, next_word):

#Este filtro es importante en este caso que no filtramos el texto (En el caso de no filtrar el texto no es neceario el if), ya que sucedía
#muchas veces que la función estaba tratando de acceder a una llave existe en el diccionario o palabra en la matriz de probabilidades. 
#Esto sucedía por palabras que incluían un come o algun signo de admiración o exclamación.

#Para corregir este error, se agregó una verificación en la función para asegurarnos de que solo intente acceder a claves existentes 
#en el diccionario P. Si la clave no existe, se devuelve un valor muy alto para que esa transición sea poco probable.
    if current_word not in P or next_word not in P[current_word]:
        return float('inf')
    return -math.log(P[current_word][next_word])


In [104]:
#Ahora creamos la función para crear texto dada una palabra inicial.
#La función recibe una "matriz" de transiciones (diccionario), una palabra inicial o semilla, el largo de palabras que se desea para el texto
#y el número de iteraciones que se desea para para explorar el espacio de estados en el Metropolis-Hasting.
#Aumentar el número de iteraciones puede mejorar la calidad del texto generado, pero también aumentará el tiempo de ejecución del algoritmo.
def generate_text_mh(P, seed_word, length, iteraciones):

#Creamos una lista donde almacenaremos las palabras que contendrá el texto.
#La variable current_word cambiará conforme vayamos agregando palabras al texto, empezando por la semilla
    current_word = seed_word
    text = [current_word]

#Recorremos el largo que queremos que sea el texto
    for i in range(length):

#Vemos si la palabra se encuentra en el diccionario, es decir, si el autor utilizaría esa palabra en alguno de sus textos
#Si no es el caso, tronamos la función
        if P.get(current_word) is None:
            print("Saavedra jamás diría eso")
            break

#Si la palabra sí está en el diccionario (matriz), comenzamos el algoritmo Metropolis-Hasting para muestrear la siguiente palabra en la cadena
#Esto lo haremos el numero de iteraciones que se desee
        for i in range(iteraciones):

#Dada la palabra en la que nos encontremos, sacaremos del diccionario las palabras que le pueden seguir como una lista, y la
#probabilidad como otra
#Recordemos que la matriz de transiciones es un diccionario de diccionarios, entonces al buscar la palabra actual current_word
#en el diccionario, nos mostrará otro diccionario con las palabras que le pueden seguir como llaves y como valores las probabilidades
            next_word_candidates = list(P[current_word].keys())
            next_word_probabilities = list(P[current_word].values())

#Teniendo la lista de palabras candidatas y probabilidades, seleccionamos una palabra utilizando la distribución propuesta 
#En este caso, proponemos distribución del modelo que sacamos anterirmente, es decir, utilizando la matriz de trancisiones (diccionario)
#También podríamos proponer que la distribución fuera uniforme para todas las palabras, pero tendría un tiempo de convergencia menor.
            proposed_next_word = random.choices(next_word_candidates, weights=next_word_probabilities, k=1)[0]

#Calculamos la razón de aceptación con la función alpha
# "Lanzamos la moneda" y vemos si nos quedamos con la palabra o nos movemos a otra

# Probabilidad de no cambiar
            a = alpha(P, current_word, text[-1])
# Probabilidad de cambio a siguiente palabra
            a2 = alpha(P, current_word, proposed_next_word)
#Calculamos la propabilidad de aceptación
            probabilidad_aceptacion = min(1, math.exp(a - a2))

# Vemos si aceptamos o rechazamos la palabra propuesta, generando valores de la distribucion Unif(0,1) y viendo si es menor a 
# Mi probabilidad de aceptación
            if random.random() < probabilidad_aceptacion:
#Si es mayor, definimos la nueva palabra
                next_word = proposed_next_word
                break
#Al terminar las iteraciones agregamos la nueva palabra a la lista
        text.append(next_word)
#Cambiamos la variable current_word a la nueva palabra agregada para seguir con el ciclo otra vez
        current_word = next_word

#Terminando el algoritmo, regresamos el texto todo junto
    return ' '.join(text)


In [105]:
# Ejemplo de uso con una palabra inicial específica
seed_word = "amarillo"
generated_text = generate_text_mh(modelo, seed_word, length=50, iteraciones=100000)
print(generated_text)

amarillo y sólo os conozco, por hecho con mucho cuando, movido a todos se llamaba Juan Rufo, jurado nada), que estáis tan buena fe se podrían reprensentallas, y los rostros son los desengaños no se contentaron de caballería y temerosas y la historia de rescate o a quien tiene usurpado; que


## Ahora limpiemos el texto a ver si tenemos convergencia más rápida

In [106]:
#Recibe una string y devuelve la string sin puntuaciones o signos raros.

def limpieza(text):
#Unidecode toma un objeto de cadena, que posiblemente contenga caracteres no ASCII, y devuelve una cadena que se puede 
#codificar de forma segura en ASCII. En este caso se utilizó para remover acentos y emojis
    text = unidecode(text)
#Minúsculas
    text = text.lower()
#Eliminar signos de interrogación, exclamación y otros
    text = re.sub(r'[^\w\s]', '', text)
#----------------------
    return text

In [107]:
file = open("El Quijote.txt", "r", encoding="utf-8")
text = file.read()
file.close()

In [108]:
text=limpieza(text)
words = text.split()
modelo = probabilidadesTransición(words)

In [109]:
# Ejemplo de uso con una palabra inicial específica
seed_word = "amarillo"
generated_text = generate_text_mh(modelo, seed_word, length=50, iteraciones=100000)
print(generated_text)

amarillo y mas deseos y diciendo ah traidor que se le parecio y guy de aquella gran reina de llamar el primero de leer de caballerias que estaban la noticia del caballo rocinante albarde el de don quijote el cual ya pasados siglos y aunque la hacienda alli tendido en ningun


## Probemos con un texto en inglés, The picture of Dorian Gray

Primero, filtrando el texto

In [110]:
file = open("Picture Of Dorian Gray.txt", "r", encoding="utf-8")
text = file.read()
file.close()

In [111]:
text=limpieza(text)
words = text.split()
modelo = probabilidadesTransición(words)

In [112]:
seed_word = "yellow"
generated_text = generate_text_mh(modelo, seed_word, length=50, iteraciones=100000)
print(generated_text)

yellow chinese box twentyseven i was nothing fearful about it all a burden to death of the women were pictured to see the emotion no you must come some strange conjectures as much better go through long _clarin_ of the room that it is like fire to linger sometimes think certainly


Ahora sin filtrar

In [113]:
with open("Picture Of Dorian Gray.txt", "r", encoding="utf-8") as file:
    content = file.read()
    words = content.split()

modelo = probabilidadesTransición(words)

In [114]:
seed_word = "yellow"
generated_text = generate_text_mh(modelo, seed_word, length=50, iteraciones=100000)
print(generated_text)

yellow piazza of evil, with myriads of myself," said Dorian. Tell me know their coats and give him the aged, I to put her with dyed hair and you really changed? Or rather, I am afraid of, I am quite obvious. But it was, but that I can it began to


## Qué tal un tweet de Donald Trump?

Probemos primero sin limpiar los tweets ... ¿qué podría salir mal?

In [115]:
import pandas as pd

In [116]:
df = pd.read_csv('tweets_01-08-2021.csv')
df

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
...,...,...,...,...,...,...,...,...,...
56566,1319485303363571714,RT @RandPaul: I don’t know why @JoeBiden think...,t,f,Twitter for iPhone,0,20683,2020-10-23 03:46:25,f
56567,1319484210101379072,RT @EliseStefanik: President @realDonaldTrump ...,t,f,Twitter for iPhone,0,9869,2020-10-23 03:42:05,f
56568,1319444420861829121,RT @TeamTrump: LIVE: Presidential Debate #Deba...,t,f,Twitter for iPhone,0,8197,2020-10-23 01:03:58,f
56569,1319384118849949702,Just signed an order to support the workers of...,f,f,Twitter for iPhone,176289,36001,2020-10-22 21:04:21,f


In [118]:
#función que transforma una columna de un dataframe en un texto plano
def colToText(df_columna):
    l=list(df_columna)
    return " ".join(l)

#lo convertimos en lista de palabras
text=colToText(df.text)
list_palabras=text.split()

In [120]:
modelo = probabilidadesTransición(list_palabras)

In [122]:
seed_word = "America"
generated_text = generate_text_mh(modelo, seed_word, length=40, iteraciones=100000)
print(generated_text)

America could extort $1,000,000.00 from the crowd. He is willing to Create Jobs, Jobs, Border than a tax cuts &amp, @gatewaypundit. @jheil at R's. Shame! ....President. We need to establish a country could be winning their best opportunity to a great
